In [561]:
# base libraries
import requests, json, os
import pandas as pd
import numpy as np
from collections import defaultdict

In [648]:
year = 2020

In [649]:
all_attributes_df = pd.read_csv (r'./output/Neighborhood_profiles_by_attribute_{}.csv'.format(year), dtype=str)

In [650]:
all_attributes_df=all_attributes_df[["Attribute", "Mission"]]

In [651]:
household_income_df=all_attributes_df[all_attributes_df.Attribute.isin(["Household Income Less than 10000",
"Household Income 10000 to 14999",
"Household Income 15000 to 19999",
"Household Income 20000 to 24999",
"Household Income 25000 to 29999",
"Household Income 30000 to 34999",
"Household Income 35000 to 39999",
"Household Income 40000 to 44999",
"Household Income 45000 to 49999",
"Household Income 50000 to 59999",
"Household Income 60000 to 74999",
"Household Income 75000 to 99999",
"Household Income 100000 to 124999",
"Household Income 125000 to 149999",
"Household Income 150000 to 199999",
"Household Income Greater than 200000"                                                                           
])]

In [652]:
median_income =all_attributes_df[all_attributes_df.Attribute.isin([
"Median Household Income (B19013_001)"                                                                            
])].to_dict('records')

In [653]:
total_households =int(float(all_attributes_df[all_attributes_df.Attribute.isin([
"Housholds"                                                                            
])].to_dict('records')[0]['Mission']))


In [654]:
median_income = float(median_income[0]['Mission'])

In [655]:
ami_breaks = [30, 50, 80, 100, 120, 150]

In [656]:
income_breaks = [(i/100) * median_income for i in ami_breaks]

In [657]:
breaks_dict = dict(zip(ami_breaks, income_breaks))

In [658]:
min_dict = {"Household Income Less than 10000":0,
"Household Income 10000 to 14999":10000,
"Household Income 15000 to 19999":15000,
"Household Income 20000 to 24999":20000,
"Household Income 25000 to 29999":25000,
"Household Income 30000 to 34999":30000,
"Household Income 35000 to 39999":35000,
"Household Income 40000 to 44999":40000,
"Household Income 45000 to 49999":45000,
"Household Income 50000 to 59999":50000,
"Household Income 60000 to 74999":60000,
"Household Income 75000 to 99999":75000,
"Household Income 100000 to 124999":100000,
"Household Income 125000 to 149999":125000,
"Household Income 150000 to 199999":150000,
"Household Income Greater than 200000":200000}       
   

max_dict = {"Household Income Less than 10000":9999,
"Household Income 10000 to 14999":14999,
"Household Income 15000 to 19999":19999,
"Household Income 20000 to 24999":24999,
"Household Income 25000 to 29999":29999,
"Household Income 30000 to 34999":34999,
"Household Income 35000 to 39999":39999,
"Household Income 40000 to 44999":44999,
"Household Income 45000 to 49999":49999,
"Household Income 50000 to 59999":59999,
"Household Income 60000 to 74999":74999,
"Household Income 75000 to 99999":99999,
"Household Income 100000 to 124999":124999,
"Household Income 125000 to 149999":149999,
"Household Income 150000 to 199999":199999,
"Household Income Greater than 200000":250000}        

In [659]:
household_income_df['min_val'] = household_income_df["Attribute"].map(min_dict)
household_income_df['max_val'] = household_income_df["Attribute"].map(max_dict)

<ipython-input-659-292e0d80ef1c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household_income_df['min_val'] = household_income_df["Attribute"].map(min_dict)
<ipython-input-659-292e0d80ef1c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household_income_df['max_val'] = household_income_df["Attribute"].map(max_dict)


In [660]:
household_income_df["Mission"]=household_income_df["Mission"].astype(float)

<ipython-input-660-4c16757f1157>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household_income_df["Mission"]=household_income_df["Mission"].astype(float)


In [661]:
household_income_dict = household_income_df.to_dict('records')

In [662]:
# deals with multiple break points in a range
split_attributes = {}
j = 0
for value in household_income_dict:
    all_breaks = []
    
    for i, break_interval in enumerate(breaks_dict):
        break_value = breaks_dict[break_interval]
        if break_value >=value['min_val'] and break_value <=value['max_val']:
            portion_below = break_interval
            
            if len(list(breaks_dict.keys()))> (i+1):
                next_interval = list(breaks_dict.keys())[i+1]
            else:
                next_interval = '150+'
                
            all_breaks.append({"break_interval":break_interval, "break_value":break_value, "below":portion_below, "above":next_interval})
        
    #for break_interval, break_value in breaks_dict.items():
    #    if break_value >=value['min_val'] and break_value <=value['max_val']:
    #        all_breaks.append({break_interval:break_value})
            
    if len(all_breaks)==1:
      
        break_value = all_breaks[0]["break_value"]
        break_interval = all_breaks[0]["break_interval"]
        above = value['max_val']-break_value
        below = break_value-value['min_val']
        portion_below = value["Mission"] * (below/(below+above))
        portion_above = value["Mission"] * (above/(below+above))
        split_attributes[j] = {}
        split_attributes[j]['attributes'] = value["Attribute"]
        split_attributes[j][all_breaks[0]["below"]] = portion_below
        split_attributes[j][all_breaks[0]["above"]] = portion_above
        j+=1
        
    if len(all_breaks)==2:
        print(all_breaks)
        break_value1 = all_breaks[0]["break_value"]
        break_interval1 = all_breaks[0]["break_interval"]
        break_value2 = all_breaks[1]["break_value"]
        break_interval2 = all_breaks[1]["break_interval"]
        above = value['max_val']-break_value2
        middle = break_value2-break_value1
        below = break_value1-value['min_val']
        portion_below = value["Mission"] * (below/(below+above+middle))
        portion_middle = value["Mission"] * (middle/(below+above+middle))
        portion_above = value["Mission"] * (above/(below+above+middle))
        split_attributes[j] = {}
        split_attributes[j]['attributes'] = value["Attribute"]
        split_attributes[j][all_breaks[0]["below"]] = portion_below
        split_attributes[j][all_breaks[0]["above"]] = portion_middle
        split_attributes[j][all_breaks[1]["above"]] = portion_above
        j+=1
     
        

split_attributes 

[{'break_interval': 120, 'break_value': 152270.80820265377, 'below': 120, 'above': 150}, {'break_interval': 150, 'break_value': 190338.51025331725, 'below': 150, 'above': '150+'}]


{0: {'attributes': 'Household Income 35000 to 39999',
  30: 198.21319511188082,
  50: 124.78680488811918},
 1: {'attributes': 'Household Income 60000 to 74999',
  50: 332.0031850132989,
  80: 1112.996814986701},
 2: {'attributes': 'Household Income 100000 to 124999',
  80: 116.27003077710543,
  100: 1803.7299692228946},
 3: {'attributes': 'Household Income 125000 to 149999',
  100: 125.50502020080759,
  120: 1532.4949797991924},
 4: {'attributes': 'Household Income 150000 to 199999',
  120: 135.16120744610157,
  150: 2265.834942754345,
  '150+': 575.0038497995534}}

In [664]:
all_split_attributes = []

In [665]:
for item in split_attributes.values():
    all_split_attributes.append(item['attributes'])


In [666]:
household_income_df["interval"] = ''
household_income_df = household_income_df[~household_income_df.Attribute.isin(all_split_attributes)]

for break_interval in breaks_dict:
    break_val = breaks_dict[break_interval]
    household_income_df["interval"] = np.where((household_income_df['max_val']<=break_val) & (household_income_df["interval"]==''), break_interval, household_income_df["interval"])

<ipython-input-666-68a49f7a994a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household_income_df["interval"] = ''


In [667]:
household_income_df['interval'] = np.where(household_income_df['interval']=='', '150+', household_income_df['interval'])

In [668]:
interval_sum = household_income_df.groupby('interval')['Mission'].sum().reset_index()

In [669]:
interval_sum

,interval,Mission
0,30,4417.0
1,50,1566.0
2,80,1812.0
3,150+,7100.0


In [670]:
split_attributes

{0: {'attributes': 'Household Income 35000 to 39999',
  30: 198.21319511188082,
  50: 124.78680488811918},
 1: {'attributes': 'Household Income 60000 to 74999',
  50: 332.0031850132989,
  80: 1112.996814986701},
 2: {'attributes': 'Household Income 100000 to 124999',
  80: 116.27003077710543,
  100: 1803.7299692228946},
 3: {'attributes': 'Household Income 125000 to 149999',
  100: 125.50502020080759,
  120: 1532.4949797991924},
 4: {'attributes': 'Household Income 150000 to 199999',
  120: 135.16120744610157,
  150: 2265.834942754345,
  '150+': 575.0038497995534}}

In [671]:
portions = {}
for values in split_attributes.values():
    for key, value in values.items():
        
        if key != 'attributes':
            if str(key) in portions:
                portions[str(key)]+=value
            else:
                portions[str(key)]=value
       

In [672]:
portions

{'30': 198.21319511188082,
 '50': 456.78998990141804,
 '80': 1229.2668457638065,
 '100': 1929.2349894237022,
 '120': 1667.656187245294,
 '150': 2265.834942754345,
 '150+': 575.0038497995534}

In [673]:
interval_sum_dic = interval_sum.to_dict("records")   

In [674]:
for record in interval_sum_dic:
    interval = str(record['interval'])
    total = record['Mission']
    portions[interval]+=total
    

In [675]:
portions

{'30': 4615.213195111881,
 '50': 2022.789989901418,
 '80': 3041.2668457638065,
 '100': 1929.2349894237022,
 '120': 1667.656187245294,
 '150': 2265.834942754345,
 '150+': 7675.003849799554}

In [676]:
portion_percentage = {}
for portion, val in portions.items():
    portion_percentage[portion] = (val/total_households)*100

In [677]:
total_120 = 0
for portion, val in portions.items():
    if portion in ['30', '50', '80', '100', '120']:
        total_120 += val
        


In [678]:
portions['total_120'] = total_120

In [679]:
portion_percentage['total_120'] = (total_120/total_households)*100

In [680]:
portion_percentage

{'30': 19.878594112554943,
 '50': 8.712538182803197,
 '80': 13.09931018548394,
 '100': 8.309579142110103,
 '120': 7.1829098817474,
 '150': 9.759378656821918,
 '150+': 33.0576898384785,
 'total_120': 57.182931504699575}

In [681]:
portion_percentage_dic = pd.DataFrame(portion_percentage.items(), columns=["ami", "percentage"])

In [682]:
download_path = r"./output"

In [683]:
portion_percentage_dic['percentage']=portion_percentage_dic['percentage'].astype("float")

In [684]:
portion_percentage_dic.to_csv(os.path.join(download_path,'amis_{}.csv'.format(year)), index = False)

In [431]:
'''all_split_dfs = []
split_attributes = {}
for i, break_interval in enumerate(breaks_dict):
    portion_below = 'portion_'+str(break_interval)
    if len(list(breaks_dict.keys()))> (i+1):
        next_interval = list(breaks_dict.keys())[i+1]
    else:
        next_interval = '150+'
    portion_above = 'portion_'+str(next_interval)
    break_val = breaks_dict[break_interval]
    household_income_df_split = household_income_df[(household_income_df['max_val']>=break_val) & (household_income_df['min_val']<=break_val)]
    print(household_income_df_split)
    household_income_df_split['break_val'] = break_val
    household_income_df_split['above'] = household_income_df_split['max_val']-break_val
    household_income_df_split['below'] = break_val-household_income_df_split['min_val']
    household_income_df_split[portion_below] = household_income_df_split["Mission"]*(household_income_df_split['below']/(household_income_df_split['below']+household_income_df_split['above']))
    household_income_df_split[portion_above] = household_income_df_split["Mission"]*(household_income_df_split['above']/(household_income_df_split['below']+household_income_df_split['above']))
    all_split_dfs.append(household_income_df_split)
    split_attributes[i] = {}
    split_attributes[i]['attributes'] = household_income_df_split['Attribute'].iloc[0]
    split_attributes[i][portion_below] = household_income_df_split[portion_below].iloc[0]
    split_attributes[i][portion_above] = household_income_df_split[portion_above].iloc[0]'''
    

                          Attribute  Mission  min_val  max_val
87  Household Income 35000 to 39999    348.0    35000    39999
                          Attribute  Mission  min_val  max_val
91  Household Income 60000 to 74999   1359.0    60000    74999
                            Attribute  Mission  min_val  max_val
93  Household Income 100000 to 124999   1702.0   100000   124999
                            Attribute  Mission  min_val  max_val
94  Household Income 125000 to 149999   1390.0   125000   149999
                            Attribute  Mission  min_val  max_val
95  Household Income 150000 to 199999   2801.0   150000   199999
                            Attribute  Mission  min_val  max_val
95  Household Income 150000 to 199999   2801.0   150000   199999


<ipython-input-431-339b8db6a446>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household_income_df_split['break_val'] = break_val
<ipython-input-431-339b8db6a446>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household_income_df_split['above'] = household_income_df_split['max_val']-break_val
<ipython-input-431-339b8db6a446>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do